## Initialization

In [8]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
import seaborn as sns
import torch

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
import itertools
palette = itertools.cycle(sns.color_palette())

# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## Import Data

In [9]:
# Read the CSV file into a pandas DataFrame and display the first few rows
path_ella = 'C:/Users/ellad/Desktop/MB_ML/Projekt/train_heart.csv'
path_train = './train_heart.csv'
path_test = './test_heart.csv'
df_train = pd.read_csv(path_train)
df_train.head()
df_test = pd.read_csv(path_test)
df_test.head()

#_________Training____________
mat = df_train.values
X_train = mat[:,0:-1]
print('x_train shape: ',X_train.shape)
y_train = mat[:,-1].astype("int")
print('y_train shape: ',y_train.shape)
ind = mat[:,1].astype("int")  #and get the indexes
#print(ind.shape)

# standardize input features
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)
X_train = (X_train - X_mean) / X_std
#print(X_train.shape)

print(' ')
#_________Test____________
mat = df_test.values
X_test = mat[:,0:-1]
print('x_test shape: ',X_test.shape)
y_test = mat[:,-1].astype("int")
print('x_test shape: ',y_test.shape)
ind = mat[:,1].astype("int")  #and get the indexes
#print(ind.shape)

# standardize input features
X_mean = X_test.mean(axis=0)
X_std = X_test.std(axis=0)
X_test = (X_test - X_mean) / X_std
#print(X_test.shape)

x_train shape:  (1025, 13)
y_train shape:  (1025,)
 
x_test shape:  (303, 13)
x_test shape:  (303,)


## NN Model

In [10]:
from pyro.nn import PyroModule, PyroSample
import torch.nn as nn
class FFNN(PyroModule):
    def __init__(self, n_in, n_hidden, n_out):
        super(FFNN, self).__init__()
        
        # Architecture
        self.in_layer = PyroModule[nn.Linear](n_in, n_hidden)
        self.in_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_hidden, n_in]).to_event(2))

        self.h_layer = PyroModule[nn.Linear](n_hidden, n_hidden)
        self.h_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_hidden, n_hidden]).to_event(2))

        self.out_layer = PyroModule[nn.Linear](n_hidden, n_out)
        self.out_layer.weight = PyroSample(dist.Normal(0., 1.).expand([n_out, n_hidden]).to_event(2))

        # Activation functions
        self.tanh = nn.Tanh()
        
    def forward(self, X, y=None):
        X = self.tanh(self.in_layer(X))
        X = self.tanh(self.h_layer(X))
        X = self.out_layer(X)
        prediction_mean = X.squeeze(-1)

        """
        with pyro.plate("observations"):
            y = pyro.sample("obs", dist.Categorical(logits=alpha + X.matmul(beta)), obs=y)
        """

        with pyro.plate("observations"):
            y = pyro.sample("obs", dist.Bernoulli(logits=prediction_mean), obs=y)
            
        return y

In [11]:
X = torch.tensor(X_train).float()
y = torch.tensor(y_train).float()


In [12]:
# Define guide function
model = FFNN(n_in=X.shape[1], n_hidden=4, n_out=1)
guide = AutoDiagonalNormal(model)
pyro.clear_param_store()

In [17]:
# Define the number of optimization steps
n_steps = 10000

# Setup the optimizer
adam_params = {"lr": 0.005}
optimizer = Adam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X, y)
    if step % 1000 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 331.7
[1000] ELBO: 328.7
[2000] ELBO: 334.5
[3000] ELBO: 328.6
[4000] ELBO: 343.6
[5000] ELBO: 340.4
[6000] ELBO: 331.7
[7000] ELBO: 337.9
[8000] ELBO: 324.3
[9000] ELBO: 315.6


In [18]:
# Prepare test data for Pyro
X_test = torch.tensor(X_test).float()

from pyro.infer import Predictive
predictive = Predictive(model, guide=guide, num_samples=1000,
                        return_sites=("obs", "_RETURN"))
samples = predictive(X_test)

samples = samples['obs'].detach().squeeze()
y_hat = np.round(samples.mean(axis=0).numpy())

mae = np.mean(np.abs(y_test - y_hat))
print("MAE:", mae)
print("Accuracy:", 1.0*np.sum((y_hat) == y_test) / len(y_test))


/var/folders/rl/2d6x2yy927z8k4n7s2v1ng3m0000gn/T/ipykernel_80657/3383048768.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test).float()


MAE: 0.07260726072607261
Accuracy: 0.9273927392739274
